## Imports

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix, classification_report

from tqdm import tqdm

## Configuration

In [3]:
# Set the plotting style to seaborn instead of matplotlib
sns.set_theme()

# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

if device == 'cuda':
    torch.cuda.empty_cache()

Using device: cuda
